# Boosted ttH multi-tagger

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import keras
from keras import utils as np_utils
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from root_pandas import read_root
from IPython.display import clear_output
from collections import OrderedDict
from random import randint
import random
from collections import Counter

In [ ]:
#Read in principle ROOT
global data
data = read_root(infile, treename, columns=inputvariables)

### Feature Separation
- data_train_scaled spat out above^ is the required data set, with variable scaled between (-1,1)
- No OHE performed, eta retained.

\begin{align}
\text{separation} = \frac{1}{2} \sum_{i \in bins} \frac{(S_{i}-B_{i})^{2}}{S_{i}+B_{i}} \\
\end{align}

where $S_{i}$ and $B_{i}$ are bin counts for signal/background respectively. This is performed for e.g. S=Higgs, B=Top on a jet-by-jet basis across the whole training set distribution for each variable to give separation scores that can be compared feature-by-feature.

In [ ]:
#Set feature_selection_2 to True above..

if feature_selection_2:
    N = 100
    bins = np.linspace(-1,1,N)
    Num_features = len(data_train_scaled[0].columns.values.tolist())
    histograms = []

    #for i in data_train_scaled[0].columns:
    #manually set order to match up with Garson's plot
    for i in ['subjet2_pt','subjet1_mv2','subjet2_mv2','d12','Qw','m','d23','nsubjets','tau32','subjet1_pt','eta','tau21','D2']:
        histograms.append((np.histogram(data_train_scaled[0][i].values.tolist(),bins)[0].tolist(),
                           np.histogram(data_train_scaled[1][i].values.tolist(),bins)[0].tolist(),
                           np.histogram(data_train_scaled[2][i].values.tolist(),bins)[0].tolist()))

def GetSeparation(S,B):
    T_S,T_B = sum(S),sum(B)
    S = np.array([float(i)/float(T_S) for i in S])
    B = np.array([float(i)/float(T_B) for i in B])
    
    separation = 0.0
    for i in range(len(S)):
        if S[i] + B[i] >0.0:
            separation += 0.5 * (((S[i]-B[i])**2)/(S[i]+B[i]))
    return separation

if feature_selection_2:
    os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/RandomFigures")  
    indices = [(1,0),(2,0),(1,2)]
    results = [[],[],[]]
    for j in indices:
        for i in range(Num_features):
            results[indices.index(j)].append(GetSeparation(histograms[i][j[0]],histograms[i][j[1]]))

    separation_results = results[2]
    X = range(Num_features)
    w = 0.2
    
    plt.figure()
    plt.title("Feature Ranking by separation")
    plt.bar([i-w for i in X],results[0],width=w,label = "S=Higgs, B=QCD")
    plt.bar(X,results[1],width=w,label="S=Top,B=QCD")
    plt.bar([i+w for i in X],results[2],width=w,label="S=Higgs,B=Top")
    plt.xticks(X,['subjet2_pt','subjet1_mv2','subjet2_mv2','d12','Qw','m','d23','nsubjets','tau32','subjet1_pt','eta','tau21','D2'],rotation=90)
    plt.ylabel("Feature Separation")
    plt.xlabel("Feature")
    plt.legend()
    plt.savefig("Feature_separation_combined", bbox_inches='tight')
    plt.show()



### Binary Test (Top and Higgs) for new feature ranking
builds binary classifier (top/Higgs only) with single hidden layer, to apply Garson's algorithm to rank features.

In [ ]:
binary = False
import os
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
if binary:
    os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/")  
    def build_binary(data_train):
        num_features = len(data_train[0])
        model = Sequential()
        model.add(Dense(10, activation="relu", input_dim=num_features))
        #model.add(Dropout(0.5))
        #model.add(Dense(100, activation="relu"))
        #model.add(Dropout(0.5))
        #model.add(Dense(1000, activation="relu"))

        model.add(Dense(2, activation="softmax"))

        #print the model architecture
        model.summary()
        return model,num_features

    model2, num_features = build_binary(data_train)
    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []
            self.fig = plt.figure()
            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            #self.acc.append(logs.get('acc'))
            #self.val_acc.append(logs.get('val_acc'))
            self.i += 1
            #f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

            clear_output(wait=True)

            plt.plot(self.x, self.losses, label="loss")
            #plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();

    plot_losses2 = PlotLosses()

    
    flag_train = np.array([[i[1],i[2]] for i in flag_train])
    
    learning_rate = 0.01
    nEpochs = 70
    decay_rate = learning_rate/nEpochs
    decay_rate = 1e-6
    momentum = 0.2
    sgd = SGD(lr=learning_rate,momentum=momentum,decay=decay_rate,nesterov=False)
    #sgd = SGD(lr=learning_rate)
    model2.compile(optimizer=sgd, loss="categorical_crossentropy")

    model2.fit(data_train, flag_train,epochs=nEpochs, callbacks=[plot_losses2], verbose=1)
    
    def garsons(A,B):
        B=np.diag(B)
        cw = np.dot(A,B)
        cw_h = abs(cw).sum(axis=0)
        rc = np.divide(abs(cw),abs(cw_h))
        rc = rc.sum(axis=1)
        ri = rc/rc.sum()
        labels = features_tested.tolist()


        plt.figure()
        plt.bar(labels,ri)
        plt.ylabel("rel importance")
        plt.xticks(rotation=90)
        plt.title("Relative Feature Importance")
        plt.savefig('garson.png')
        plt.show()
        
        return ri


    r = garsons(model2.get_weights()[0],model2.get_weights()[1])
    R = sum(separation_results)
    separation_results = [float(i)/float(R) for i in separation_results] #normalise to one.
    

    plt.figure()
    plt.bar(X,separation_results,width=w,label="TMVA Feature Separation")
    plt.bar([i+w for i in X], r, width=w,label="Garson's (using Higgs/Top binary classifier)")
    plt.xlabel("Feature")
    plt.xticks(X,['subjet2_pt','subjet1_mv2','subjet2_mv2','d12','Qw','m','d23','nsubjets','tau32','subjet1_pt','eta','tau21','D2'],rotation=90)
    plt.ylabel("Importance")
    plt.title("Comparison of Feature Ranking Methods")
    plt.legend()
    plt.savefig("ranking_comparison",bbox_inches='tight')
    plt.show()
    
    
    

    flag_test = np.array([[i[1],i[2]] for i in flag_test])
    predictions_binary = model2.predict_proba(data_test)
    plt.figure()
    for i,j,k in [([i[0] for i in flag_test],[i[0] for i in predictions_binary],"Higgs"),([i[1] for i in flag_test],[i[1] for i in predictions_binary],"Top")]:
        fpr, tpr, thresholds = roc_curve(i, j)
        auc_val = auc(fpr, tpr)
        plt.plot(fpr, tpr, label='{} (area = {:.3f})'.format(k,auc_val))

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()


## Train the model

First we define the network architecture. The first layer must specify the shape of the inputs. The choice of n dense layers here is somewhat arbitrary and can be altered. <br>
The output layer defines 3 classes and uses softmax which assigns decimal probabilities to each class in the problem and constrains those probabilities to sum to 1.
<font color='red'>NOTE: careful with input dimension if fiddling with adding new parameters at top that aren't being used as features.</font> 



In [ ]:
def build(data_train):
    num_features = len(data_train[0])
    model = Sequential()
    model.add(Dense(1000, activation="relu", input_dim=num_features))
    #model.add(Dropout(0.5))
    model.add(Dense(1000, activation="relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(1000, activation="relu"))

    model.add(Dense(3, activation="softmax"))

    #print the model architecture
    model.summary()
    return model,num_features

model,num_features = build(data_train)


We want to be able to monitor the loss function during training. Make a plot which updates after each epoch:

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        #self.acc.append(logs.get('acc'))
        #self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        #f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        plt.plot(self.x, self.losses, label="loss")
        #plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
class PlotWeights(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.fig = plt.figure()
        self.logs = []
        self.i = 0
    def on_epoch_end(self,epoch, logs={}):
        self.logs.append(logs)
        
        clear_output(wait=True)
        self.i += 1
          
        w = []
        for i in model.get_weights()[2]:
            for j in i:
                w.append(j)
        
        plt.hist(w)
        plt.show();       

plot_losses = PlotLosses()
plot_weights = PlotWeights()

Set some tuneable parameters:
- learning rate
- number of epochs
- loss function
- ....

And train the model!

### Learning Rate/ Momentum and Decay Rate
Momentum allows algo to remember its last step, and apply some 'intertia' to the gradient descent - doesn't get stuck in brief flat regions if they exist
decay rate lowers 'step-size' over time to supposedly improve convergence if tuned correctly

Can easily test without these extra parameters by unhashing the sgd=SGD(lr) line.


In [ ]:
learning_rate = 0.01
nEpochs = 70
decay_rate = learning_rate/nEpochs
decay_rate = 1e-6
momentum = 0.2
sgd = SGD(lr=learning_rate,momentum=momentum,decay=decay_rate,nesterov=False)
#sgd = SGD(lr=learning_rate)
model.compile(optimizer=sgd, loss="categorical_crossentropy")

model.fit(data_train, flag_train,epochs=nEpochs, callbacks=[plot_losses], verbose=1)

## Evaluation

Grab the predictions and have a look at a few:

In [ ]:
print data_train.shape
print data_test.shape

print data_test[:10]
print data2[:10]
print data2.shape

In [ ]:
def predict_outcome(data_test,model):
    #save the weights
    model.save_weights("weights.h5")
    predictions = model.predict_proba(data_test)
    return predictions

predictions = predict_outcome(data_test,model)
predictions_ttbar = predict_outcome(data2,model)
print predictions
print "Total Number of predictions: ", len(predictions)
print "Total Number of unique predictions: ",  len([list(x) for x in set(tuple(x) for x in predictions)])

## Outputs
Work out 'score' per event, how good NN is at classifying within each event, rather than for each large-R jet.
Bundle subjets into events. Then find subjet with highest Higgs score to compare with flag for that subjet.

In [ ]:
os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/RandomFigures")  

#return dictionary with key=eventNumber, value=list of all predictions within that event [(QCD1,Higgs1,Top1),(QCD2,Higgs2,Top2)...etc]
def event_dictionary(predictions,maps,list_evs):
    count = 0
    events = OrderedDict((event, []) for (event,j) in maps.values())
    for prediction in predictions:
        events[list_evs[count][0]].append((prediction,count)) 
        count +=1
    return events

#Return dictionary with key=eventNumber, value = list[ all predictions,count (an index variable), NBjets ]
def event_dictionary_withsubBjets(predictions,maps,list_evs,subBjets):
    count = 0
    events = OrderedDict((event, []) for (event,j) in maps.values())
    for prediction in predictions:
        events[list_evs[count][0]].append((prediction,count,subBjets[count])) 
        count +=1
    return events

#find all maxima (if more than one) - shouldn't be necessary anymore.
def multi_max(list_vals,ind):
    my_max = max(map(lambda x: x,list_vals))
    return [x for x in list_vals if x==my_max]

def err(count,total):
    a = count
    c = total-count
    return np.sqrt((c/(a**2))*(1/((1+(c/a))**3)))
     
def event_score(classifier,events_dict,flags):
    event_predictions = []
    indices = {"qcd":0,"Higgs":1,"top":2}
    ind = indices[classifier] 
    predictions_events = []
    for event in events_dict.keys():
        if len(events_dict[event]) != 0: #filter out events in training set
            index_max = [jet[0][ind] for jet in events_dict[event]].index(max([jet[0][ind] for jet in events_dict[event]]))
            index = events_dict[event][index_max][1]
            flag_subjet = flags[index][ind]
            event_predictions.append(flag_subjet)
            predictions_events.append(max([jet[0][ind] for jet in events_dict[event]]))
    print "Fraction of correct", classifier, "tagging by event: ", float(np.sum(event_predictions))/float(len(event_predictions)),"+/-", err(float(np.sum(event_predictions)),float(len(event_predictions)))
    return predictions_events,event_predictions, float(np.sum(event_predictions))/float(len(event_predictions))
   
    
    
events = event_dictionary(predictions,mapping,list_event_nos)
eventsttb = event_dictionary(predictions_ttbar,mapping2,list_event_nos2)    

    
from collections import Counter
#Look at predicted composition of events
#Take each jet within an event, count number of tops and higgs predicted by each event.

def composition(eventss,title):
    scatter_info = []
    for event in eventss.keys():
        if len(eventss[event]) > 1:
            num_higgs = 0
            num_top = 0 
            for jet in eventss[event]:
                jet = jet[0]
                ind_max = np.argmax(jet)
                if ind_max == 1:
                    num_higgs +=1
                if ind_max == 2:
                    num_top +=1

            scatter_info.append((num_higgs,num_top))



    results =  Counter(scatter_info)
    x = [i[0] for i in results.keys()]
    y = [i[1] for i in results.keys()]
    sizes = results.values()*200

    plt.figure()
    plt.title(title)
    plt.scatter(x, y, s=sizes, alpha=0.3,
                cmap='viridis')
    for i in range(len(x)):
        plt.annotate(sizes[i],results.keys()[i],ha="center")
    plt.grid()
    plt.xlabel("Num (Higgs)")
    plt.ylabel("Num (Top)")
    plt.savefig("EventComposition_{}".format(title))
    plt.show()
    

#Do the same thing for the truth flagging - understand how the events are 'really' composed...
events_truth = event_dictionary(flag_test,mapping,list_event_nos)
composition(events,"Predicted_event_composition")
composition(events_truth,"Actual_event_composition")
     
    
from operator import itemgetter
from matplotlib.colors import LogNorm


print "P(higgs) v.s. P(top) for non-QCD tagged jets"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) == 0] #Remove QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))


def probab_plotter(x,y,xlab,ylab,title):
    plt.figure()
    plt.hist2d(x,y,bins=[np.linspace(0,1,15),np.linspace(0,1,15)],norm=LogNorm())
    plt.colorbar()
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.savefig(title)
    plt.show()


### A whole load of probability correlations
    

print "P(higgs) v.s. P(top) for only non-QCD tagged jets"
P_higgs = [i[1] for i in predictions_sorted]
P_top = [i[2] for i in predictions_sorted]
probab_plotter(P_higgs,P_top,"P(Higgs)","P(Top)","Higgs_Top_nonQCDtagged")

print "P(higgs) v.s. P(top) for only QCD tagged jets"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) != 0] #Keep only QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_top = [i[2] for i in predictions_sorted]
probab_plotter(P_higgs,P_top,"P(Higgs)","P(Top)","Higgs_TOP_QCDtagged")

print "P(higgs) v.s. P(top) for only Higgs tagged jets"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) != 1] #Keep only QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_top = [i[2] for i in predictions_sorted]
probab_plotter(P_higgs,P_top,"P(Higgs)","P(Top)","Higgs_TOP_Higgs_Tagged")

print "P(higgs) v.s. P(top) for only top tagged jets"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) != 2] #Keep only QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_top = [i[2] for i in predictions_sorted]
probab_plotter(P_higgs,P_top,"P(Higgs)","P(Top)","Higgs_TOP_top_tagged")

print "P(higgs) v.s. P(QCD) (all jets)"
p = predictions.tolist()
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_higgs,P_qcd,"P(Higgs)","P(qcd)","Higgs_QCD")

print "P(higgs) v.s. P(QCD) (non QCD Jets)"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) == 0] #Remove QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_higgs,P_qcd,"P(Higgs)","P(qcd)","Higgs_QCD_nonQCDjets")

print "P(higgs) v.s. P(QCD) (Higgs Jets)"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) != 1] #Remove QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(1))
P_higgs = [i[1] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_higgs,P_qcd,"P(Higgs)","P(qcd)","Higgs_QCD_Higgsjets")

print "P(top) v.s. P(QCD) (all jets)"
p = predictions.tolist()
predictions_sorted = sorted(p,key=itemgetter(2))
P_top = [i[2] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_top,P_qcd,"P(top)","P(qcd)","Top_QCD")

print "P(top) v.s. P(QCD) (nonQCD jets)"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) == 0] #Remove QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(2))
P_top = [i[2] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_top,P_qcd,"P(top)","P(qcd)","Top_QCD_nonQCDjets")

print "P(top) v.s. P(QCD) (Top jets)"
p = predictions.tolist()
[p.remove(i) for i in p if np.argmax(i) != 2] #Remove QCD tagged jets
predictions_sorted = sorted(p,key=itemgetter(2))
P_top = [i[2] for i in predictions_sorted]
P_qcd = [i[0] for i in predictions_sorted]
probab_plotter(P_top,P_qcd,"P(top)","P(qcd)","Top_QCD_topjets")
 
    
    
#Selection algorithms to deal with ambiguity
    
    
#Give Higgs priority
ev_pred_higgs, ev_pred_top = [],[]
for event in events.keys():
        if len(events[event]) >1:
            index_higgs = [jet[0][1] for jet in events[event]].index(max([jet[0][1] for jet in events[event]]))
            checker = events[event][index_higgs][1]
            index_top = [jet[0][2] for jet in events[event]].index(max([jet[0][2] for jet in events[event] if jet[1] != checker]))
            index_higgs = events[event][index_higgs][1]
            index_top = events[event][index_top][1]
            flag_subjet_higgs = flag_test[index_higgs][1]
            flag_subjet_top = flag_test[index_top][2]
            ev_pred_higgs.append(flag_subjet_higgs)
            ev_pred_top.append(flag_subjet_top)
    
print "Giving Higgs priority...."
print "Higgs", float(np.sum(ev_pred_higgs))/float(len(ev_pred_higgs))
print "Top", float(np.sum(ev_pred_top))/float(len(ev_pred_top))
            
#Give Top priority
ev_pred_higgs, ev_pred_top = [],[]
for event in events.keys():
        if len(events[event]) >1:
            index_top = [jet[0][2] for jet in events[event]].index(max([jet[0][2] for jet in events[event]]))
            checker = events[event][index_top][1]
            index_higgs = [jet[0][1] for jet in events[event]].index(max([jet[0][1] for jet in events[event] if jet[1] != checker]))
            index_higgs = events[event][index_higgs][1]
            index_top = events[event][index_top][1]
            flag_subjet_higgs = flag_test[index_higgs][1]
            flag_subjet_top = flag_test[index_top][2]
            ev_pred_higgs.append(flag_subjet_higgs)
            ev_pred_top.append(flag_subjet_top)
    
print "Giving top priority...."
print "Higgs", float(np.sum(ev_pred_higgs))/float(len(ev_pred_higgs))
print "Top", float(np.sum(ev_pred_top))/float(len(ev_pred_top))
  
#Finding which jet has the highest value in other jets outside the one they overlap on.
ev_pred_higgs, ev_pred_top = [],[]
for event in events.keys():
        if len(events[event]) >1:
            index_top = [jet[0][2] for jet in events[event]].index(max([jet[0][2] for jet in events[event]]))
            index_higgs = [jet[0][1] for jet in events[event]].index(max([jet[0][1] for jet in events[event]]))
            checker=events[event][index_top][1]
            if index_top == index_top:
                max_other_higgs = max([jet[0][1] for jet in events[event] if jet[1] != checker])
                max_other_top = max([jet[0][2] for jet in events[event] if jet[1] != checker])
                if max_other_higgs >max_other_top:
                    index_higgs = [jet[0][1] for jet in events[event]].index(max_other_higgs)
                else:
                    index_top = [jet[0][2] for jet in events[event]].index(max_other_top)
                
            index_higgs = events[event][index_higgs][1]
            index_top = events[event][index_top][1]
            flag_subjet_higgs = flag_test[index_higgs][1]
            flag_subjet_top = flag_test[index_top][2]
            ev_pred_higgs.append(flag_subjet_higgs)
            ev_pred_top.append(flag_subjet_top)
    
print "Using secondary jets...."
print "Higgs", float(np.sum(ev_pred_higgs))/float(len(ev_pred_higgs))
print "Top", float(np.sum(ev_pred_top))/float(len(ev_pred_top))
            
    
#Just take whichever one is highest within that jet
ev_pred_higgs, ev_pred_top = [],[]
for event in events.keys():
        if len(events[event]) >1:
            index_top = [jet[0][2] for jet in events[event]].index(max([jet[0][2] for jet in events[event]]))
            index_higgs = [jet[0][1] for jet in events[event]].index(max([jet[0][1] for jet in events[event]]))
            checker=events[event][index_top][1]
            if index_top == index_top:
                max_higgs = max([jet[0][1] for jet in events[event]])
                max_top = max([jet[0][2] for jet in events[event]])
                if max_higgs > max_top:
                    index_top = [jet[0][2] for jet in events[event]].index(max([jet[0][2] for jet in events[event] if jet[1] != checker]))
                else:
                    index_higgs = [jet[0][1] for jet in events[event]].index(max([jet[0][1] for jet in events[event] if jet[1] != checker]))
    
            index_higgs = events[event][index_higgs][1]
            index_top = events[event][index_top][1]
            flag_subjet_higgs = flag_test[index_higgs][1]
            flag_subjet_top = flag_test[index_top][2]
            ev_pred_higgs.append(flag_subjet_higgs)
            ev_pred_top.append(flag_subjet_top)
    
print "Using max in first jet...."
print "Higgs", float(np.sum(ev_pred_higgs))/float(len(ev_pred_higgs))
print "Top", float(np.sum(ev_pred_top))/float(len(ev_pred_top))


print "Doing nothing..."

output_here = True # set to true to print stuff out
if output_here:
    pred_ev_higgs,flag_ev_higgs,sc = event_score("Higgs",events,flag_test)
    pred_ev_top,flag_ev_top,sc = event_score("top",events,flag_test)
    pred_ev_qcd,flag_ev_qcd,sc = event_score("qcd",events,flag_test)


## Receiver Operating Characteristic (ROC)
For binary classifiers - visualise performance, whilst Area under the Curve (AUC) gives a value between 0 and 1 (the higher the better).

For classifiers with >2 classes - use 'one v.s. all approach': yielding 3 ROC curves. For each class, we use the other two classes jointly as the negative class (i.e. use a classification in the other 2 classes as a false positive).

 - class 1 v.s. classes 2&3
 - class 2 v.s. classes 1&3
 - class 3 v.s. classes 1&2
 
 Two slightly different metrics to evaluate overall performance.
 'Micro-averaging' and 'Macro-averaging' invovle using individual true positives, false positives etc, and a average of individual class performance respectively.
 
 Exact definition used from (ref):
 In “micro averaging”, we’d calculate the performance, e.g., precision, from the individual true positives (TP), true negatives, false positives (FP), and false negatives of the k-class model:
 
 \begin{align}
 \text{Generally, precision}, \epsilon = \frac{TP}{TP+FP}, \\
 \text{In micro-averaging: }, \epsilon_{micro} = \frac{ \sum_{i=1}^{k} TP_{i}}{\sum_{i=1}^{k} (TP_{i} +FP_{i})} \\
 \text{In macro-averaging: },\epsilon_{macro} = \frac{\sum_{i=1}^{k} \epsilon_{i}}{k} \\
 \end{align}
 
 (Ref: https://www.dlology.com/blog/simple-guide-on-how-to-generate-roc-plot-for-keras-classifier/ ).
 
 
 

In [ ]:
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc

def ROC():

    n_classes=3

    # Plot linewidth.
    lw = 2

    y_test = flag_test
    y_score = predictions

    classes = {0:"qcd",1:"Higgs",2:"top"}


    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(1)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(classes[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for multi-class')
    plt.legend(loc="lower right")
    plt.savefig("ROC.png")
    plt.show()


    # Zoom in view of the upper left corner.
    plt.figure(2)
    plt.xlim(0, 0.3)
    plt.ylim(0.4, 1)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(classes[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for multi-class')
    plt.legend(loc="upper left")
    plt.savefig("zoomedROC.png")
    plt.show()

ROC_plot = False
if ROC_plot:
    ROC()

## ROC based on event-scoring. 
- Each time, treat as a binary classifier where only outcome of interest is whether or not each event gets the top,qcd or HIggs right respectively.
- ONLY WORKS if test set is ttH (i.e. all events are signal). Not if ttbar set included in test set)

In [ ]:
plt.figure()
for i,j,k in [(flag_ev_higgs,pred_ev_higgs,"Higgs"),(flag_ev_top,pred_ev_top,"Top"),(flag_ev_qcd,pred_ev_qcd,"QCD")]:
    fpr, tpr, thresholds = roc_curve(i, j)
    auc_val = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='{} (area = {:.3f})'.format(k,auc_val))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



## Signal/ Background Selection Plots using jet tagging information 

### Algorithm

For each event:
\begin{align}
\text{count number of jets for which P(H) is max prob: } N_{H} \\
\text{count number of jets for which P(T) is max prob: } N_{T} \\
\end{align}

If $N_{H}$ > 0:

\begin{align}
\text{Take jet with max P(H) to be the Higgs Jet } \\
\end{align}


And If $N_{T}$ > 0:

\begin{align}
\text{Take jet with max P(T) to be the Top Jet} \\
\end{align}


Any other permutations are rejected (i.e. not signal).
Cuts are also performed based on b-tagging information, either NBjets >3, or NadditionalsubBjets $\geq 1$.


In [ ]:


## New Method for application to Loose selection (with event weights)
from matplotlib.colors import LogNorm
import os
os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/")  
os.chdir("2Dplots")

if new_loose:
    events_subb = event_dictionary_withsubBjets(predictions,mapping,list_event_nos,events_and_NsubBjets)
    eventsttb_subb = event_dictionary_withsubBjets(predictions_ttbar,mapping2,list_event_nos2,events_and_NsubBjets_background)   

NBins = 20
bins_input = np.linspace(0,1.0,NBins)

print "Number of predictions: ", len(predictions)
print "Number of signal events total: ", len(events.keys())
print "Number of background events total: ", len(eventsttb.keys())
print "Number of predictions (background): ", len(predictions_ttbar)

#Signal
scatter_values = []
for key,value in events.iteritems():
    w = events_and_weights[key]
    if len(value) >1:
        max_probs = [np.argmax(i[0]) for i in value]
        count_higgs = 0
        count_top = 0
        for i in max_probs:
            if i ==1.0:
                count_higgs +=1
            if i== 2.0:
                count_top +=1
        if count_higgs == 1:
            higgs = value[max_probs.index(1)][0][1]
            if count_top==1:
                top = value[max_probs.index(2)][0][2]
                scatter_values.append((higgs,top,w))
            if count_top>1:
                top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                scatter_values.append((higgs,top,w))
        if count_higgs >1:
            higgs = max([i[1] for i in value])
            if count_top == 1:
                top = value[max_probs.index(2)][0][2]
                scatter_values.append((higgs,top,w))
            if count_top >1:
                top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                scatter_values.append((higgs,top,w))
                
print "Number of signal events: ", len(scatter_values)

plt.figure()
plt.title("Signal: Loose Selection P(top) v.s P(Higgs) for sorted events")
S_nocut = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_signal",bbox_inches='tight')
plt.show()
                
#Signal - but with NBjets cut.
scatter_values = []
for key,value in events.iteritems():
    if events_and_NBjets[key] >3:
        w = events_and_weights[key]
        if len(value) >1:
            max_probs = [np.argmax(i[0]) for i in value]
            count_higgs = 0
            count_top = 0
            for i in max_probs:
                if i ==1.0:
                    count_higgs +=1
                if i== 2.0:
                    count_top +=1
            if count_higgs == 1:
                higgs = value[max_probs.index(1)][0][1]
                if count_top==1:
                    top = value[max_probs.index(2)][0][2]
                    scatter_values.append((higgs,top,w))
                if count_top>1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    scatter_values.append((higgs,top,w))
            if count_higgs >1:
                higgs = max([i[1] for i in value])
                if count_top == 1:
                    top = value[max_probs.index(2)][0][2]
                    scatter_values.append((higgs,top,w))
                if count_top >1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    scatter_values.append((higgs,top,w))
    
print "Number of signal events (post NBjet cut): ", len(scatter_values)

plt.figure()
plt.title("Signal: Loose Selection P(top) v.s P(Higgs) for sorted events with NBjets Cut")
S_cut = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_signal_NBJETCUT",bbox_inches='tight')
plt.show()



if new_loose:
    #Signal - new b-tag search
    scatter_values = []
    for key,value in events_subb.iteritems():
        w = events_and_weights[key]
        NBjets = events_and_NBjets[key]
        if len(value) >1:
            max_probs = [np.argmax(i[0]) for i in value]
            count_higgs = 0
            count_top = 0
            for i in max_probs:
                if i ==1.0:
                    count_higgs +=1
                if i== 2.0:
                    count_top +=1
            if count_higgs == 1:
                higgs = value[max_probs.index(1)][0][1]
                subb_higgs = value[max_probs.index(1)][2]
                if count_top==1:
                    top = value[max_probs.index(2)][0][2]
                    subb_top = value[max_probs.index(2)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
                if count_top>1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    subb_top = value[[value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)].index(top)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
            if count_higgs >1:
                higgs = max([i[1] for i in value])
                subb_higgs = value[[i[1] for i in value].index(higgs)][2]
                if count_top == 1:
                    top = value[max_probs.index(2)][0][2]
                    subb_top = value[max_probs.index(2)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
                if count_top >1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    subb_top = value[[value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)].index(top)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))

    print "Number of signal events (post subBjet cut): ", len(scatter_values)

    plt.figure()
    plt.title("Signal: Loose Selection P(top) v.s P(Higgs) for sorted events (single b-tag)")
    S_cut_new = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_weighted_loose_signal_BTAGCUTSINGLE",bbox_inches='tight')
    plt.show()

#Background
scatter_values = []
for key,value in eventsttb.iteritems():
    w = events_and_weights_background[key]
    if len(value) >1:
        max_probs = [np.argmax(i[0]) for i in value]
        count_higgs = 0
        count_top = 0
        for i in max_probs:
            if i ==1.0:
                count_higgs +=1
            if i== 2.0:
                count_top +=1
        if count_higgs == 1:
            higgs = value[max_probs.index(1)][0][1]
            if count_top==1:
                top = value[max_probs.index(2)][0][2]
                scatter_values.append((higgs,top,w))
            if count_top>1:
                top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                scatter_values.append((higgs,top,w))
        if count_higgs >1:
            higgs = max([i[1] for i in value])
            if count_top == 1:
                top = value[max_probs.index(2)][0][2]
                scatter_values.append((higgs,top,w))
            if count_top >1:
                top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                scatter_values.append((higgs,top,w))

print "Number of background events: ", len(scatter_values)
                
plt.figure()
plt.title("Background: Loose Selection P(top) v.s P(Higgs) for sorted events")
B_nocut = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_background",bbox_inches='tight')
plt.show()
                
#Background - but with NBjets cut.
scatter_values = []
for key,value in eventsttb.iteritems():
    if events_and_NBjets_background[key] >3:
        w = events_and_weights_background[key]
        if len(value) >1:
            max_probs = [np.argmax(i[0]) for i in value]
            count_higgs = 0
            count_top = 0
            for i in max_probs:
                if i ==1.0:
                    count_higgs +=1
                if i== 2.0:
                    count_top +=1
            if count_higgs == 1:
                higgs = value[max_probs.index(1)][0][1]
                if count_top==1:
                    top = value[max_probs.index(2)][0][2]
                    scatter_values.append((higgs,top,w))
                if count_top>1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    scatter_values.append((higgs,top,w))
            if count_higgs >1:
                higgs = max([i[1] for i in value])
                if count_top == 1:
                    top = value[max_probs.index(2)][0][2]
                    scatter_values.append((higgs,top,w))
                if count_top >1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    scatter_values.append((higgs,top,w))

                    
print "Number of background events (post nBjet cut: ", len(scatter_values)
plt.figure()
plt.title("Background: Loose Selection P(top) v.s P(Higgs) for sorted events with NBjets Cut")
B_cut = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_background_NBJETSCUT",bbox_inches='tight')
plt.show()

if new_loose:
    #Background - new b-tag search
    scatter_values = []
    for key,value in eventsttb_subb.iteritems():
        w = events_and_weights_background[key]
        NBjets = events_and_NBjets_background[key]
        if len(value) >1:
            max_probs = [np.argmax(i[0]) for i in value]
            count_higgs = 0
            count_top = 0
            for i in max_probs:
                if i ==1.0:
                    count_higgs +=1
                if i== 2.0:
                    count_top +=1
            if count_higgs == 1:
                higgs = value[max_probs.index(1)][0][1]
                subb_higgs = value[max_probs.index(1)][2]
                if count_top==1:
                    top = value[max_probs.index(2)][0][2]
                    subb_top = value[max_probs.index(2)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
                if count_top>1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    subb_top = value[[value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)].index(top)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
            if count_higgs >1:
                higgs = max([i[1] for i in value])
                subb_higgs = value[[i[1] for i in value].index(higgs)][2]
                if count_top == 1:
                    top = value[max_probs.index(2)][0][2]
                    subb_top = value[max_probs.index(2)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
                if count_top >1:
                    top = max([value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)])
                    subb_top = value[[value[i][0][2] for i in range(len(value)) if i != max_probs.index(1)].index(top)][2]
                    if NBjets - (subb_higgs+subb_top) >=1:
                        scatter_values.append((higgs,top,w))
                        
    print "Number of background events (post subBjet cut: ", len(scatter_values)
    plt.figure()
    plt.title("Background: Loose Selection P(top) v.s P(Higgs) for sorted events (single b-tag)")
    B_cut_new = plt.hist2d([i[0] for i in scatter_values],[i[1] for i in scatter_values],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_weighted_loose_background_BTAGCUTSINGLE",bbox_inches='tight')
    plt.show()


S_over_B_nocut = S_nocut[0]/B_nocut[0]
S_over_rootB_nocut = S_nocut[0]/np.sqrt(B_nocut[0])

#And same for set with b-tagging cut.
S_over_B_cut = S_cut[0]/B_cut[0]
S_over_rootB_cut = S_cut[0]/np.sqrt(B_cut[0])

if new_loose:
    #And same for set with new single b-tagging cut.
    S_over_B_cutnew = S_cut_new[0]/B_cut_new[0]
    S_over_rootB_cutnew = S_cut_new[0]/np.sqrt(B_cut_new[0])

    
bin_centres = ["{0:.2f}".format((bins_input[i] + bins_input[i+1])/2.0) for i in range(NBins-1)]


plt.figure()
plt.title("S/B - no b-cut")
plt.imshow(S_over_B_nocut.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_SoverB",bbox_inches='tight')
plt.show()

plt.figure()
plt.title("S/B - b-cut")
plt.imshow(S_over_B_cut.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_SoverB_CUT",bbox_inches='tight')
plt.show()

if new_loose:
    plt.figure()
    plt.title("S/B - b-cut new")
    plt.imshow(S_over_B_cutnew.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_weighted_loose_SoverB_CUTNEW",bbox_inches='tight')
    plt.show()

plt.figure()
plt.title(r"S/\sqrt{B} - no b-cut")
plt.imshow(S_over_rootB_nocut.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_SoverRootB",bbox_inches='tight')
plt.show()

plt.figure()
plt.title(r"S/\sqrt{B} - b-cut")
plt.imshow(S_over_rootB_cut.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.colorbar()
plt.savefig("2D_weighted_loose_SoverRootB_CUT",bbox_inches='tight')
plt.show()

if new_loose:
    plt.figure()
    plt.title(r"S/\sqrt{B} - b-cut NEW")
    plt.imshow(S_over_rootB_cutnew.T,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_weighted_loose_SoverRootB_CUTNEW",bbox_inches='tight')
    plt.show()

### Weighted sum of signal and background events

In [ ]:
print "Actual num events, (sum of weights): Signal, no cut: ", np.sum(S_nocut[0])
print "Actual num events, (sum of weights): Signal, NBjets cut: ", np.sum(S_cut[0])
print "Actual num events, (sum of weights): Signal, additional b jets cut: ", np.sum(S_cut_new[0])
print "Actual num events, (sum of weights): Background, no cut: ", np.sum(B_nocut[0])
print "Actual num events, (sum of weights): Background, NBjets cut:", np.sum(B_cut[0])
print "Actual num events, (sum of weights): Background, additiona; b jets cut:", np.sum(B_cut_new[0])

## Selection Parameters contour plots
- Take 'signal' if following critera are met:
\begin{align}
\text{if: P(Higgs)} \geq h \text{ for } \geq 1 \text{ jet} \\
\text{and, P(top)} \geq t \text{ for} \geq 1 \text{ jet} \\
\end{align}
- Take background if not
- Perform grid search over 2D parameter space (t,h)

In [ ]:
#Find events which have a true Higgs
new_thing = data.groupby('eventNumber')['MatchFlag'+matching].value_counts().unstack().fillna(0)
higgs_events = new_thing[new_thing[1] == 1] # look at events which contain a True higgs.
events_with_higgs = higgs_events.index.values.tolist()

In [ ]:
from matplotlib.colors import LogNorm
import os
os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/")  
os.chdir("2Dplots")

#When choosing max(P(x)) for events, must make sure that these don't correspond to the SAME jet.
#Therefore a choice has to be made about priorities - for the moment chose P(top) to be most imporant,
#given the models performance is usually better on tops

background_scatter_info = []
k = 0
tot = 0
k1 = 0
tot1 = 0
for key,value in eventsttb.iteritems():
    if len(value) >1:
        higgs = max([i[0][1] for i in value])
        num_h = value[[i[0][1] for i in value].index(higgs)][1]
        top = max([i[0][2] for i in value])
        num_t = value[[i[0][2] for i in value].index(top)][1]
        if num_t == num_h:
            background_scatter_info.append((higgs,top))
            k +=1
        tot +=1
amb_back = float(k)/float(tot)
    
print "Number of ambiguous events (background):", k, "out of", tot

scatter_info = []
k = 0
tot = 0
k1 = 0
tot1 = 0
for key,value in events.iteritems():
    if len(value) >1:
        higgs = max([i[0][1] for i in value])
        num_h = value[[i[0][1] for i in value].index(higgs)][1]
        top = max([i[0][2] for i in value])
        num_t = value[[i[0][2] for i in value].index(top)][1]
        if num_t == num_h:
            k +=1
            scatter_info.append((higgs,top))
        tot +=1
    
print "Number of ambiguous events (signal):", k, "out of", tot
amb_sig = float(k)/float(tot)

#Work out same information for signal, but this time only use events which contain a True Higgs.
new_scatter_info = []
k = 0
tot = 0
k1 = 0
tot1 = 0
for key,value in events.iteritems():
    if key in events_with_higgs:
        if len(value) >1:
            higgs = max([i[0][1] for i in value])
            num_h = value[[i[0][1] for i in value].index(higgs)][1]
            top = max([i[0][2] for i in value])
            num_t = value[[i[0][2] for i in value].index(top)][1]
            if num_t == num_h:
                k +=1
                new_scatter_info.append((higgs,top))
            tot +=1
    
print "Number of ambiguous events (signal):", k, "out of", tot
amb_sig = float(k)/float(tot)

plt.figure()
plt.hist2d([i[0] for i in background_scatter_info],[i[1] for i in background_scatter_info],bins=[np.linspace(0,1,15),np.linspace(0,1,15)],norm=LogNorm())
plt.colorbar()
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.title("P(Higgs) v.s. P(Top) for ambiguous jets")
plt.savefig("Background_Ambiguous_jets")
plt.show()
  
    
plt.figure()
plt.hist2d([i[0] for i in scatter_info],[i[1] for i in scatter_info],bins=[np.linspace(0,1,15),np.linspace(0,1,15)],norm=LogNorm())
plt.colorbar()
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.title("P(Higgs) v.s. P(Top) for ambiguous jets")
plt.savefig("Ambiguous_jets")
plt.show()

plt.figure()
plt.hist2d([i[0] for i in new_scatter_info],[i[1] for i in new_scatter_info],bins=[np.linspace(0,1,15),np.linspace(0,1,15)],norm=LogNorm())
plt.colorbar()
plt.xlabel("P(Higgs)")
plt.ylabel("P(Top)")
plt.title("P(Higgs) v.s. P(Top) for ambiguous jets")
plt.savefig("Ambiguous_jets_events_containing_True_Higgs_only")
plt.show()

def do_contours_2():
    signal = []
    for key,value in events.iteritems():
        if len(value) > 1: #more than 1 large R jet
            higgs = max([i[0][1] for i in value])
            num_h = value[[i[0][1] for i in value].index(higgs)][1]
            top = max([i[0][2] for i in value if i[1] != num_h])

            signal.append((top,higgs))

    background = []
    for key,value in eventsttb.iteritems():
        if len(value) >1:
            higgs = max([i[0][1] for i in value])
            num_h = value[[i[0][1] for i in value].index(higgs)][1]
            top = max([i[0][2] for i in value if i[1] != num_h])

            background.append((top,higgs))
    print len(signal),len(background)



    x = np.linspace(0,1,10)
    y = [(x[j]+x[j+1])/2.0 for j in range(len(x)-1)]



    titles = ["signal","background","S/B",r"S/\sqrt{B}"]
    output = [signal,background]

    plt.figure()
    plt.subplot(2,2,1)
    plt.tight_layout()
    plt.title("signal")
    s = plt.hist2d([i[0] for i in signal],[i[1] for i in signal],bins = [x,x])
    plt.xlabel("P(top)")
    plt.ylabel("P(Higgs)")
    plt.colorbar()
    plt.subplot(2,2,2)
    plt.tight_layout()
    plt.title("background")
    b = plt.hist2d([i[0] for i in background],[i[1] for i in background],bins = [x,x])    
    plt.xlabel("P(top)")
    plt.ylabel("P(Higgs)")    
    plt.colorbar()
    S_over_B = s[0]/b[0]
    S_over_B_r = s[0]/np.sqrt(b[0])
    X,Y = np.meshgrid(y,y)
    plt.subplot(2,2,3)
    plt.tight_layout()
    plt.title(r"S/B")
    plt.imshow(S_over_B.T,origin='lower')
    plt.xlabel("P(top)")
    plt.ylabel("P(Higgs)")    
    plt.colorbar()
    plt.subplot(2,2,4)
    plt.tight_layout()
    plt.title(r'S/ \sqrt{B}')
    plt.imshow(S_over_B_r.T,origin='lower')
    plt.xlabel("P(top)")
    plt.ylabel("P(Higgs)")    
    plt.colorbar()
    plt.savefig('newplots_2dcontour')
    plt.show()


### New Selection idea - cutting on P(Higgs) & P(Top)

In [ ]:
#Plot 1D histograms
#Max P(H) in event and max P(T) in event
#Could also plot this on 2D histogram right..
os.chdir("/home/ppe/d/dgould/tth-dnn-tagger/python/2Dplots")  

signal_h,signal_t,background_h,background_t = [],[],[],[]

#DONT DO THE B-Tag cut thing
for event in events.keys():
    if len(events[event]) >1:
        signal_h.append(max([i[0][1] for i in events[event]]))
        signal_t.append(max([i[0][2] for i in events[event]]))
for event in eventsttb.keys():
    if len(eventsttb[event]) >1:
        background_h.append(max([i[0][1] for i in eventsttb[event]]))
        background_t.append(max([i[0][2] for i in eventsttb[event]]))
    
# DO THE B-Tag cut thing
extra=True
if extra:
    signal_h,signal_t,backgound_h,backgound_t = [],[],[],[]

    for key,event in events_subb.iteritems():
        if len(event) >1:
            higgs = max([i[0][1] for i in events[key]])
            top = max([i[0][2] for i in events[key]])
            NBjets = events_and_NBjets[key]

            subb_higgs = event[[i[0][1] for i in events_subb[key]].index(higgs)][2]
            subb_top = event[[i[0][2] for i in events_subb[key]].index(top)][2]
            ind1 = [i[0][1] for i in events_subb[key]].index(higgs)
            ind2 = [i[0][2] for i in events_subb[key]].index(top)
            if NBjets - (subb_higgs+subb_top) >=1 or ind1 == ind2:
                   signal_h.append(higgs)
                   signal_t.append(top)

    for key,event in eventsttb_subb.iteritems():
        if len(event):
            higgs = max([i[0][1] for i in eventsttb_subb[key]])
            top = max([i[0][2] for i in eventsttb_subb[key]])
            NBjets = events_and_NBjets_background[key]

            subb_higgs = event[[i[0][1] for i in eventsttb_subb[key]].index(higgs)][2]
            subb_top = event[[i[0][2] for i in eventsttb_subb[key]].index(top)][2]      
            
            ind1 = [i[0][1] for i in eventsttb_subb[key]].index(higgs)
            ind2 = [i[0][2] for i in eventsttb_subb[key]].index(top)
            if NBjets - (subb_higgs+subb_top) >=1 or ind1 == ind2:
                   background_h.append(higgs)
                   background_t.append(top)





                
plt.figure()
plt.hist(signal_h)
plt.title("Signal")
plt.xlabel("Max P(Higgs) in event")
plt.savefig("1Dhiggssignal")
plt.show()

                
plt.figure()
plt.hist(signal_h)
plt.title("Signal")
plt.xlabel("Max P(Higgs) in event")
plt.yscale("log")
plt.savefig("1DhiggssignalLOG")
plt.show()

plt.figure()
plt.title("Signal")
plt.hist(signal_t)
plt.xlabel("Max P(Top) in event")
plt.savefig("1Dtopsignal")
plt.show()


plt.figure()
plt.title("Background")
plt.hist(background_h)
plt.xlabel("Max P(Higgs) in event")
plt.savefig("1Dhiggsbackground")
plt.show()


plt.figure()
plt.title("Background")
plt.hist(background_h)
plt.xlabel("Max P(Higgs) in event")
plt.yscale("log")
plt.savefig("1DhiggsbackgroundLOG")
plt.show()

plt.figure()
plt.title("Background")
plt.hist(background_t)
plt.xlabel("Max P(Top) in event")
plt.savefig("1Dtopbackground")
plt.show()

x = np.linspace(0,1,10)
plt.figure()
plt.title("Signal")
plt.hist2d(signal_t,signal_h,bins = [x,x])
plt.xlabel("P(top)")
plt.ylabel("P(Higgs)")
plt.colorbar()
plt.savefig("2Dnewsignal",bbox_inches='tight')
plt.show()

plt.figure()
plt.title("Background")
plt.hist2d(background_t,background_h,bins = [x,x])
plt.xlabel("P(top)")
plt.ylabel("P(Higgs)")
plt.colorbar()
plt.savefig("2Dnewbackgound",bbox_inches='tight')
plt.show()
    

In [ ]:
#Cutting points
h = 0.0
t = 0.0

scatter_values_signal = []



def do_signal(h,t):
    scatter_values_signal = []
    count_N = 0
    count_1 = 0
    overlap = 0
    for key,value in events_subb.iteritems():
        w = events_and_weights[key]
        NBjets = events_and_NBjets[key]
        if len(value) >1:
            #higgs_jets = [i[0][1] for i in value if i[0][1] >= h]
            top_jets = [i[0][2] for i in value if i[0][2] >= t]

            #if len(higgs_jets) >0:
            if len(top_jets) >0:
                #don't deal with ambiguity
                #top_jets = [i[0][2] for i in value if i[0][2] >= t]
            
                #deal with ambiguity by giving Higgs priority 
                #top_jets = [i[0][2] for i in value if i[0][2] >= t if i[0][1] != max(higgs_jets)]
                #higgs_jets = [i[0][1] for i in value if i[0][1] >= h if i[0][2] != max(top_jets)]
                higgs_jets = [i[0][1] for i in value if i[0][1] >=h]
                #if len(top_jets) >0:
                if len(higgs_jets) >0: 
                    higgs = max(higgs_jets)
                    top = max(top_jets)
                    ind1 = [i[0][1] for i in events_subb[key]].index(higgs)
                    ind2 = [i[0][2] for i in events_subb[key]].index(top)
                    subb_higgs = value[[i[0][1] for i in events_subb[key]].index(higgs)][2]
                    subb_top = value[[i[0][2] for i in events_subb[key]].index(top)][2]
                    #print NBjets - (subb_higgs+subb_top),ind1,ind2
                    if ind1 == ind2:
                        overlap +=1

                    #if NBjets - (subb_higgs+subb_top) >=1 or ind1 == ind2:
                    if NBjets - (subb_higgs+subb_top) >=1 and ind1 != ind2:
                        scatter_values_signal.append((higgs,top,w))
                    else:
                        count_N += w
                    
        else:
            count_1 +=w
                    
    if len(scatter_values_signal) !=0:             
        return scatter_values_signal,count_N,count_1,float(overlap)/float(len(scatter_values_signal))
    else:
        return scatter_values_signal,count_N,count_1,0.0

        
def do_back(h,t):
    count_N = 0
    count_1 = 0
    overlap = 0
    scatter_values_background = []
    for key,value in eventsttb_subb.iteritems():
        w = events_and_weights_background[key]
        NBjets = events_and_NBjets_background[key]
        if len(value) >1:
            #higgs_jets = [i[0][1] for i in value if i[0][1] >= h]
            top_jets = [i[0][2] for i in value if i[0][2] >=t]
  
            #if len(higgs_jets) >0:
            if len(top_jets) >0:
                #don't deal with ambiguitiy
                #top_jets = [i[0][2] for i in value if i[0][2] >= t]
                #Give Higgs priority 
                #top_jets = [i[0][2] for i in value if i[0][2] >= t if i[0][1] != max(higgs_jets)]
                #higgs_jets = [i[0][1] for i in value if i[0][1] >=h if i[0][2] != max(top_jets)]
                higgs_jets = [i[0][1] for i in value if i[0][1] >=h]

                #if len(top_jets) >0:
                if len(higgs_jets) >0:
                    higgs = max(higgs_jets)
                    top = max(top_jets)
                    ind1 = [i[0][1] for i in eventsttb_subb[key]].index(higgs)
                    ind2 = [i[0][2] for i in eventsttb_subb[key]].index(top)
                    subb_higgs = value[[i[0][1] for i in eventsttb_subb[key]].index(higgs)][2]
                    subb_top = value[[i[0][2] for i in eventsttb_subb[key]].index(top)][2]
                    if ind1 == ind2:
                        overlap +=1
                    if NBjets - (subb_higgs+subb_top) >=1 and ind1 != ind2:
                    #if NBjets - (subb_higgs+subb_top) >=1 or ind1==ind2:
                        scatter_values_background.append((higgs,top,w))

                    else:
                        count_N += w

        else:
            count_1 +=w
                 
    if len(scatter_values_background) !=0:
        return scatter_values_background,count_N,count_1,float(overlap)/float(len(scatter_values_background))
    else:
        return scatter_values_background,count_N,count_1,0.0

            
scatter_values_signal = do_signal(h,t)
scatter_values_background = do_back(h,t)

print scatter_values_signal[1:],np.sum([i[2] for i in scatter_values_signal[0]])
print scatter_values_background[1:],np.sum([i[2] for i in scatter_values_background[0]])

plotting_stuff = False
if plotting_stuff:
    plt.figure()
    plt.title("New Cutting: Signal Plot")
    S_new = plt.hist2d([i[0] for i in scatter_values_signal],[i[1] for i in scatter_values_signal],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values_signal])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_signal_NEWIDEA",bbox_inches='tight')
    plt.show()


    plt.figure()
    plt.title("New Cutting: Background Plot")
    B_new = plt.hist2d([i[0] for i in scatter_values_background],[i[1] for i in scatter_values_background],bins=[bins_input,bins_input],weights=[i[2] for i in scatter_values_background])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_background_NEWIDEA",bbox_inches='tight')
    plt.show()

    SoverB_new = S_new[0]/B_new[0]
    SrootB_new = S_new[0]/np.sqrt(B_new[0])

    plt.figure()
    plt.title("New Cutting: S/B")
    plt.imshow(SoverB_new,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_SoverB_NEWIDEA",bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.title("New Cutting: S/root(B)")
    plt.imshow(SrootB_new,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("P(Higgs)")
    plt.ylabel("P(Top)")
    plt.colorbar()
    plt.savefig("2D_SrootB_NEWIDEA",bbox_inches='tight')
    plt.show()

    print np.sum(S_new[0])
    print np.sum(B_new[0])

    print sum(events_and_weights.values())


    
big_plots = True
if big_plots:
    N = 10
    param = np.linspace(0,1.0,N).tolist()
    output_s = np.zeros((N,N))
    output_b = np.zeros((N,N))
    output_sb = np.zeros((N,N))
    output_srb = np.zeros((N,N))
    output_ov = np.zeros((N,N))
    output_ovb = np.zeros((N,N))

    for x in param:
        for y in param:
            sig,a,b,overlap_s = do_signal(x,y)
            bac,a,b,overlap_b = do_back(x,y)
            S = plt.hist2d([i[0] for i in sig],[i[1] for i in sig],bins=[bins_input,bins_input],weights=[i[2] for i in sig])
            B = plt.hist2d([i[0] for i in bac],[i[1] for i in bac],bins=[bins_input,bins_input],weights=[i[2] for i in bac])
            output_s[param.index(x)][param.index(y)] = np.sum(S[0])
            output_b[param.index(x)][param.index(y)] = np.sum(B[0])
            output_sb[param.index(x)][param.index(y)] = np.sum(S[0])/np.sum(B[0])
            output_srb[param.index(x)][param.index(y)] = np.sum(S[0])/np.sqrt(np.sum(B[0]))
            output_ov[param.index(x)][param.index(y)] = overlap_s
            output_ovb[param.index(x)][param.index(y)] = overlap_b

            print x,y, np.sum(S[0])

    plt.figure()
    plt.title("Signal")
    plt.imshow(output_s,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("brandnew_signal",bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.title("Background")
    plt.imshow(output_b,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("brandnew_background",bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.title(r"Signal/Background")
    plt.imshow(output_sb,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("brandnew_soverb",bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.title(r"S/ \sqrt{B}")
    plt.imshow(output_srb,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("brandnew_soverrootb",bbox_inches='tight')
    plt.show()
    
    plt.figure()
    plt.title(r"Signal overlapping fraction")
    plt.imshow(output_ov,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("overlaps",bbox_inches='tight')
    plt.show()
    
    plt.figure()
    plt.title(r"Background overlapping fraction")
    plt.imshow(output_ovb,origin='lower',extent=[0.0,1.0,0.0,1.0])
    plt.xlabel("t")
    plt.ylabel("h")
    plt.colorbar()
    plt.savefig("overlapb",bbox_inches='tight')
    plt.show()

In [ ]:
c = 0
for key in events_subb.keys():
    c+= events_and_weights[key]
print c

In [ ]:
!jupyter nbconvert --to script BoostedttHTagger.ipynb
print "DONE!"